In [1]:
from mlinspect.monkeypatching._mlinspect_ndarray import MlinspectNdarray
import pandas as pd
import os
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from mlinspect.utils import get_project_root
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# prepare data for explainer
COUNTIES_OF_INTEREST = ['county2', 'county3']

patients = pd.read_csv(os.path.join(str(get_project_root()), "example_pipelines", "healthcare",
                                    "patients.csv"), na_values='?')
histories = pd.read_csv(os.path.join(str(get_project_root()), "example_pipelines", "healthcare",
                                     "histories.csv"), na_values='?')

data = patients.merge(histories, on=['ssn'])
complications = data.groupby('age_group') \
    .agg(mean_complications=('complications', 'mean'))
data = data.merge(complications, on=['age_group'])
data['label'] = data['complications'] > 1.2 * data['mean_complications']
data = data[['smoker', 'last_name', 'county', 'num_children', 'race', 'income', 'label']]
data = data[data['county'].isin(COUNTIES_OF_INTEREST)]
train_data, test_data = train_test_split(data)
y_train = train_data["label"]
y_test = test_data["label"]
X_train = train_data.drop("label", axis=1)
X_test = test_data.drop("label", axis=1)

impute_and_one_hot_encode = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])
featurisation = ColumnTransformer(transformers=[
    ("impute_and_one_hot_encode", impute_and_one_hot_encode, ['smoker', 'county', 'race']),
    ('numeric', StandardScaler(), ['num_children', 'income']),
], remainder='drop')

X_t_train: MlinspectNdarray = featurisation.fit_transform(train_data, y_train)
print(featurisation.get_feature_names_out())
X_t_test: MlinspectNdarray = featurisation.fit_transform(X_test, y_test)

['impute_and_one_hot_encode__smoker_False'
 'impute_and_one_hot_encode__smoker_True'
 'impute_and_one_hot_encode__county_county2'
 'impute_and_one_hot_encode__county_county3'
 'impute_and_one_hot_encode__race_race1'
 'impute_and_one_hot_encode__race_race2'
 'impute_and_one_hot_encode__race_race3' 'numeric__num_children'
 'numeric__income']


/opt/venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [2]:
import numpy as np

X_t_train.view(np.ndarray)

array([[ 1.        ,  0.        ,  1.        , ...,  0.        ,
         0.35891084, -0.71384592],
       [ 1.        ,  0.        ,  1.        , ...,  0.        ,
        -1.42468427, -1.1530951 ],
       [ 0.        ,  1.        ,  1.        , ...,  1.        ,
        -0.83015256, -0.06368519],
       ...,
       [ 1.        ,  0.        ,  1.        , ...,  0.        ,
         0.95344255,  0.18366568],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.54797425, -0.87925793],
       [ 0.        ,  1.        ,  1.        , ...,  0.        ,
        -0.83015256,  0.65869945]])

In [3]:
pd.DataFrame(data=X_t_train.view(np.ndarray), columns=featurisation.get_feature_names_out())

,impute_and_one_hot_encode__smoker_False,impute_and_one_hot_encode__smoker_True,impute_and_one_hot_encode__county_county2,impute_and_one_hot_encode__county_county3,impute_and_one_hot_encode__race_race1,impute_and_one_hot_encode__race_race2,impute_and_one_hot_encode__race_race3,numeric__num_children,numeric__income
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.358911,-0.713846
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.424684,-1.153095
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.830153,-0.063685
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.547974,0.285537
4,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.547974,-1.094508
...,...,...,...,...,...,...,...,...,...
212,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.547974,-1.346620
213,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.953443,-0.106513
214,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.953443,0.183666
215,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.547974,-0.879258


In [4]:
df = pd.DataFrame([X_t_test[0]], index=[y_test])
print(df.index[0])

(True,)


In [5]:
X_t_train.shape

(217, 9)

In [6]:
X_t_train[0:2]

array([[ 1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.35891084, -0.71384592],
       [ 1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        , -1.42468427, -1.1530951 ]])

In [7]:
y_train

515    False
539    False
526     True
338     True
542    False
       ...  
425    False
549     True
829    False
396    False
9       True
Name: label, Length: 217, dtype: bool

In [ ]:
df = pd.DataFrame(data=X_t_train.view(np.ndarray), columns=featurisation.get_feature_names_out())
df.dtypes

In [9]:
from features.explainability.inspections.explainability_methods_enum import ExplainabilityMethodsEnum
from features.explainability.inspections.explainer import Explainer
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Some tensorflow warnings in the pipeline we inspect
from mlinspect.utils import get_project_root

from mlinspect import PipelineInspector

EXAMPLE_PIPELINE = os.path.join(str(get_project_root()), "features", "explainability", "pipeline_with_inspection.py")

inspector_result = PipelineInspector\
    .on_pipeline_from_py_file(EXAMPLE_PIPELINE) \
    .add_required_inspection(Explainer([ExplainabilityMethodsEnum.SHAP, ExplainabilityMethodsEnum.LIME, ExplainabilityMethodsEnum.PDP, ExplainabilityMethodsEnum.ICE, ExplainabilityMethodsEnum.INTEGRATED_GRADIENTS, ExplainabilityMethodsEnum.ALE, ExplainabilityMethodsEnum.DALE, ExplainabilityMethodsEnum.DALEX], X_t_train.view(np.ndarray), X_t_test.view(np.ndarray), featurisation.get_feature_names_out(), y_test, y_train)) \
    .execute()

extracted_dag = inspector_result.dag
dag_node_to_inspection_results = inspector_result.dag_node_to_inspection_results
check_results = inspector_result.check_to_check_results

KeyboardInterrupt: 

In [ ]:
from features.explainability.inspections.explainability_methods_enum import ExplainabilityMethodsEnum
from features.explainability.inspections.explainer import Explainer
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Some tensorflow warnings in the pipeline we inspect
from mlinspect.utils import get_project_root

from mlinspect import PipelineInspector

EXAMPLE_PIPELINE = os.path.join(str(get_project_root()), "features", "explainability", "pipeline_with_inspection.py")

inspector_result = PipelineInspector\
    .on_pipeline_from_py_file(EXAMPLE_PIPELINE) \
    .add_required_inspection(Explainer([ExplainabilityMethodsEnum.DALEX], X_t_train.view(np.ndarray), X_t_test.view(np.ndarray), featurisation.get_feature_names_out(), y_test, y_train)) \
    .execute()

extracted_dag = inspector_result.dag
dag_node_to_inspection_results = inspector_result.dag_node_to_inspection_results
check_results = inspector_result.check_to_check_results

In [9]:
from mlinspect import OperatorType

relevant_node = [node for node in extracted_dag.nodes if node.operator_info.operator in {OperatorType.ESTIMATOR,}][0]
print("\n\033[1m{} ({})\033[0m\n{}\n{}".format(
    relevant_node.operator_info.operator,
    relevant_node.details.description,
    relevant_node.optional_code_info.source_code,
    relevant_node.optional_code_info.code_reference))
inspection_result = dag_node_to_inspection_results[relevant_node][Explainer([ExplainabilityMethodsEnum.SHAP, ExplainabilityMethodsEnum.LIME, ExplainabilityMethodsEnum.PDP, ExplainabilityMethodsEnum.ICE, ExplainabilityMethodsEnum.INTEGRATED_GRADIENTS, ExplainabilityMethodsEnum.ALE, ExplainabilityMethodsEnum.DALE, ExplainabilityMethodsEnum.DALEX], X_t_train.view(np.ndarray), X_t_test.view(np.ndarray), featurisation.get_feature_names_out(),y_test, y_train)]
print(inspection_result)


OperatorType.ESTIMATOR (Neural Network)
KerasClassifier(model=create_model_predict, epochs=10, batch_size=1, verbose=0, loss='binary_crossentropy',)
CodeReference(lineno=49, col_offset=13, end_lineno=49, end_col_offset=121)


KeyError: Explainer(('SHAP', 'LIME', 'PDP', 'ICE', 'INTEGRATED_GRADIENTS', 'ALE', 'DALE', 'DALEX'))

In [9]:
from mlinspect import OperatorType

relevant_node = [node for node in extracted_dag.nodes if node.operator_info.operator in {OperatorType.ESTIMATOR,}][0]
print("\n\033[1m{} ({})\033[0m\n{}\n{}".format(
    relevant_node.operator_info.operator,
    relevant_node.details.description,
    relevant_node.optional_code_info.source_code,
    relevant_node.optional_code_info.code_reference))
inspection_result = dag_node_to_inspection_results[relevant_node][Explainer([ExplainabilityMethodsEnum.DALEX,], X_t_train.view(np.ndarray), X_t_test.view(np.ndarray), featurisation.get_feature_names_out(),y_test, y_train)]
print(inspection_result)


OperatorType.ESTIMATOR (Neural Network)
KerasClassifier(model=create_model_predict, epochs=10, batch_size=1, verbose=0, loss='binary_crossentropy',)
CodeReference(lineno=49, col_offset=13, end_lineno=49, end_col_offset=121)
{<ExplainabilityMethodsEnum.DALE: 'Differential Accumulated Local Effects'>: {'explainer': <features.explainability.dale.dale.DALE object at 0xffff20e57220>, 'results': (array([[ 0.1216374 , -0.08017011, -0.08017011, ...,  0.1216374 ,
        -0.0785939 , -0.08017011],
       [-0.08017011,  0.1216374 ,  0.1216374 , ...,  0.1216374 ,
         0.02970027,  0.1216374 ],
       [-0.08017011,  0.1216374 ,  0.1216374 , ...,  0.1216374 ,
         0.02970027,  0.1216374 ],
       ...,
       [-0.08017011,  0.1216374 , -0.08017011, ...,  0.1216374 ,
        -0.0785939 ,  0.07777841],
       [ 0.1216374 , -0.08017011, -0.08017011, ..., -0.08017011,
        -0.08017011,  0.1216374 ],
       [-0.08017011,  0.1216374 ,  0.1216374 , ...,  0.1216374 ,
         0.02970027,  0.1216

In [ ]:
inspection_result[ExplainabilityMethodsEnum.LIME]["results"].show_in_notebook()

In [ ]:
inspection_result[ExplainabilityMethodsEnum.PDP]["results"].plot()

In [ ]:
inspection_result[ExplainabilityMethodsEnum.ICE]["results"].plot()

In [9]:
inspection_result[ExplainabilityMethodsEnum.INTEGRATED_GRADIENTS]["results"].attributions

[array([[ 0.58602172, -0.        ,  0.        , -0.00244259,  0.06884654,
          0.        ,  0.        ,  0.07110923, -0.24794298],
        [ 0.28144685, -0.        ,  0.06489754, -0.        ,  0.        ,
          0.18989129,  0.        , -0.05386274,  0.1662618 ],
        [ 0.4392305 , -0.        , -0.        , -0.11221118,  0.        ,
          0.34177711,  0.        , -0.26132837,  0.2215555 ],
        [ 0.        , -0.45768301,  0.00913482, -0.        ,  0.15659924,
          0.        ,  0.        , -0.21276138,  0.26572797],
        [ 0.63043527, -0.        ,  0.        , -0.00611766,  0.07594116,
          0.        ,  0.        ,  0.01241132, -0.35129184],
        [ 0.30324198, -0.        ,  0.        ,  0.00838489,  0.        ,
          0.20008168,  0.        ,  0.00521487,  0.11601001],
        [ 0.34455755, -0.        ,  0.27145664,  0.        , -0.03561468,
          0.        ,  0.        , -0.11411897,  0.10638017],
        [ 0.50596127, -0.        ,  0.01913431, 

In [10]:
inspection_result[ExplainabilityMethodsEnum.ALE]["results"]

Explanation(meta={
  'name': 'ALE',
  'type': ['blackbox'],
  'explanations': ['global'],
  'params': {
              'check_feature_resolution': True,
              'low_resolution_threshold': 10,
              'extrapolate_constant': True,
              'extrapolate_constant_perc': 10.0,
              'extrapolate_constant_min': 0.1,
              'min_bin_points': 4}
            ,
  'version': '0.9.4'}
, data={
  'ale_values': [array([[-0.10355331,  0.10355331],
       [ 0.10355331, -0.10355331]]), array([[ 0.14958886, -0.14958886],
       [-0.14958886,  0.14958886]]), array([[-0.01952479,  0.01952479],
       [ 0.01952479, -0.01952479]]), array([[-0.01977502,  0.01977502],
       [ 0.01977502, -0.01977502]]), array([[ 0.00359291, -0.00359291],
       [-0.00359291,  0.00359291]]), array([[-0.02655177,  0.02655177],
       [ 0.02655177, -0.02655177]]), array([[ 0.00672761, -0.00672761],
       [-0.00672761,  0.00672761]]), array([[ 0.00500257, -0.00500257],
       [ 0.01633159, -0.01

In [ ]:
inspection_result[ExplainabilityMethodsEnum.DALE]["explainer"].plot()

In [ ]:
inspection_result[ExplainabilityMethodsEnum.DALEX]["results"]["train"].plot()

In [ ]:
inspection_result[ExplainabilityMethodsEnum.DALEX]["results"]["test"].plot()